In [1]:
import numpy as np 
import pandas as pd 

spam_data = pd.read_csv('/Users/emilyvincett/Downloads/spam.csv')

#Codes instances of spam as 1 and not spam as 0
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)


from sklearn.model_selection import train_test_split 
X_train, X_test,y_train,y_test = train_test_split(spam_data['text'],
                                                 spam_data['target'],
                                                 random_state=0)


In [2]:
def answer_one():
    
    spam = spam_data[spam_data['target']==1]
    return len(spam)/len(spam_data['target'])*100
answer_one()

13.406317300789663

In [3]:
def answer_two():
    
    from sklearn.feature_extraction.text import CountVectorizer

    Name = []
    Length = []
    vect = CountVectorizer().fit(X_train)
    features = vect.get_feature_names()

    for name in features: 
        length = len(name)
        Length.append(length)
        Name.append(name)
    combi = list(zip(Name,Length))
    return sorted(combi,key=lambda tup:(-tup[1],-tup[1]))[0][0]

answer_two()

'com1win150ppmx3age16subscription'

In [4]:
def answer_three():
    
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score
    from sklearn.feature_extraction.text import CountVectorizer

    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    mnb = MultinomialNB(alpha=0.1)
    mnb.fit(X_train_vectorized, y_train)
    X_test_vectorized = vect.transform(X_test)
    predictions = mnb.predict(X_test_vectorized)

    return print('Model AUC Score is %.3f' %(roc_auc_score(y_test,predictions)))
answer_three()

Model AUC Score is 0.972


In [5]:
def answer_four():
    
    from sklearn.feature_extraction.text import TfidfVectorizer

    #Creates an instance of the TfidfVectorizer and stores it in a variable named vectorizer
    vectorizer = TfidfVectorizer()

    #Transforms the training set based on the vectorizer applied into numbers.
    #Stores into a matrix like format
    tfidf = vectorizer.fit_transform(X_train)
    names = vectorizer.get_feature_names()
    maximum_tfidf = tfidf.max(0).toarray().tolist()

    #Builds a dataframe with columns as feature names and the rows as the number of documents being considered
    matrix_df = pd.DataFrame(tfidf.toarray(),columns=names)
    smallest = matrix_df.max(axis=0).sort_values(ascending=True)[:20]
    largest = matrix_df.max(axis=0).sort_values(ascending=False)[:20]
    return (smallest,largest)

answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

In [6]:
#Alternative way of collective the smallest and largest max tfidf

from sklearn.feature_extraction.text import TfidfVectorizer


vect = TfidfVectorizer().fit(X_train)

feature_names = np.array(vect.get_feature_names())

X_train_vectorized = vect.transform(X_train)

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
feature_names[sorted_tfidf_index][:-21:-1]


array(['146tf150p', 'havent', 'home', 'okie', 'thanx', 'er', 'anything',
       'lei', 'nite', 'yup', 'thank', 'ok', 'where', 'beerage', 'anytime',
       'too', 'done', '645', 'tick', 'blank'], dtype='<U32')

In [7]:
def answer_five():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score

    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    mnb = MultinomialNB(alpha=0.1)
    mnb.fit(X_train_vectorized,y_train)
    predictions = mnb.predict(vect.transform(X_test))

    return roc_auc_score(y_test,predictions)
answer_five()

0.9416243654822335

In [8]:
def answer_six():
    spam_data_3 = spam_data.copy()
    spam_data_3['length'] = spam_data_3['text'].astype(str).apply(len)
    no_spam = spam_data_3[spam_data_3['target']==0].length.mean()

    is_spam = spam_data_3[spam_data_3['target']==1].length.mean()
    return (no_spam,is_spam)
answer_six()

(71.02362694300518, 138.8661311914324)

In [9]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [10]:
def answer_seven():
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.svm import SVC 
    from sklearn.metrics import roc_auc_score

    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)

    len_x_train = X_train.apply(len)
    len_x_test = X_test.apply(len)

    X_train_add = add_feature(X_train_vectorized,len_x_train)
    X_test_add = add_feature(X_test_vectorized,len_x_test)

    clf = SVC(C=10000)
    clf.fit(X_train_add,y_train)
    predictions = clf.predict(X_test_add)

    return roc_auc_score(y_test,predictions)
answer_seven()

0.9661689557407943

In [11]:
def answer_six_alternative():
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score

    spam_data_2 = spam_data.copy()
    len(spam_data_2['text'][0])

    not_spam = spam_data_2[spam_data_2['target']==0]
    is_spam = spam_data_2[spam_data_2['target']==1]
    not_spam.index

    notspam_length = []
    for i in not_spam.index:
        l = (len(not_spam['text'][i]))
        notspam_length.append(l)
    notspam_length = np.array(notspam_length)
    notspam_length.mean()

    isspam_length = []
    for i in is_spam.index:
        l = len(is_spam['text'][i])
        isspam_length.append(l)
    isspam_length = np.array(isspam_length)
    return (notspam_length.mean(),isspam_length.mean())

answer_six_alternative()

(71.02362694300518, 138.8661311914324)

In [12]:
def answer_eight():
    
    spam_data_2 = spam_data.copy()
    spam_data_2['length'] = spam_data_2['text'].apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    return spam_data_2[spam_data_2['target']==0].length.mean(), spam_data_2[spam_data_2['target']==1].length.mean()
answer_eight()

(0.2992746113989637, 15.759036144578314)

In [13]:
def answer_nine():
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score

    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3))
    vect.fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)

    #length of documents
    len_x_train = X_train.apply(len)
    len_x_test = X_test.apply(len)

    # number of digits 
    dig_x_train = X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    dig_x_test = X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))

    X_train_add = add_feature(X_train_vectorized,[len_x_train,dig_x_train])
    X_test_add = add_feature(X_test_vectorized, [len_x_test,dig_x_test])

    clf = LogisticRegression(C=100)
    clf.fit(X_train_add,y_train)
    predictions = clf.predict(X_test_add)

    return roc_auc_score(y_test,predictions)
answer_nine()

/Users/emilyvincett/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9809793219360643

In [14]:
def answer_ten():
    
    spam_data['non_char_length']=spam_data['text'].str.findall(r'(\W)').str.len()
    return spam_data[spam_data['target']==0].non_char_length.mean(),spam_data[spam_data['target']==1].non_char_length.mean()
answer_ten()    

(17.29181347150259, 29.041499330655956)

In [15]:
def answer_eleven():
    
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score

    vect = CountVectorizer(min_df=5, ngram_range=(2,5),analyzer='char_wb')
    vect.fit(X_train)
    vect.fit(X_test)

    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)

    #add length 
    len_x_train = X_train.apply(len)
    len_x_test = X_test.apply(len)

    #digit count 
    dig_x_train = X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    dig_x_test = X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))

    #non word char count 
    non_word_x_train = X_train.str.findall(r'(\W)').str.len()
    non_word_x_test = X_test.str.findall(r'(\W)').str.len()

    X_train_add = add_feature(X_train_vectorized, [len_x_train,dig_x_train,non_word_x_train,])
    X_test_add = add_feature(X_test_vectorized,[len_x_test,dig_x_test,non_word_x_test])

    clf = LogisticRegression(C=100)
    clf.fit(X_train_add,y_train)
    predictions = clf.predict(X_test_add)
    auc = roc_auc_score(y_test,predictions)

    feature_names = np.array(vect.get_feature_names() + ['length_of_doc','digit_count','non_word_char_count'])
    sorted_coef_index = clf.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index][:10]
    largest = feature_names[sorted_coef_index][:-11:-1]
   
    return (auc, list(smallest),list(largest))
answer_eleven()

/Users/emilyvincett/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9805612617353955,
 ['..', '. ', ' i', ' go', 'ok', ' y', '? ', 'he', 'pe', 'go'],
 ['digit_count', 'co', 'ww', 'ne', 'ia', 'xt', 'ar', 'uk', 'mob', ' x'])

In [17]:
spam_data

,text,target,non_char_length
0,"Go until jurong point, crazy.. Available only ...",0,28
1,Ok lar... Joking wif u oni...,0,11
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,33
3,U dun say so early hor... U c already then say...,0,16
4,"Nah I don't think he goes to usf, he lives aro...",0,14
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,1,38
5568,Will Ì_ b going to esplanade fr home?,0,8
5569,"Pity, * was in mood for that. So...any other s...",0,16
5570,The guy did some bitching but I acted like i'd...,0,26
